In [5]:
# 특성 메모리 사용량 감소 int 64 -> 적당한 형태로
def reduce_memory_usage(df):
    """
    데이터프레임의 메모리 사용량을 줄이기 위해 데이터 타입을 축소합니다.
    오버플로우를 방지하기 위해 각 열의 값 범위를 확인하여 적절한 데이터 타입으로 변환합니다.

    Parameters:
        df (pd.DataFrame): 데이터 타입 축소를 적용할 데이터프레임.

    Returns:
        pd.DataFrame: 데이터 타입이 축소된 데이터프레임.
    """
    import pandas as pd
    import numpy as np
    
    for col in df.columns:
        col_type = df[col].dtype

        if pd.api.types.is_integer_dtype(col_type):
            # 정수형 처리
            col_min = df[col].min()
            col_max = df[col].max()

            if col_min >= -32768 and col_max <= 32767:
                df[col] = df[col].astype("int16")
            elif col_min >= -2147483648 and col_max <= 2147483647:
                df[col] = df[col].astype("int32")
            # 값이 더 클 경우, int64 유지

        elif pd.api.types.is_float_dtype(col_type):
            # 부동소수점 처리
            col_min = df[col].min()
            col_max = df[col].max()

            if col_max - col_min < 65504:
                df[col] = df[col].astype("float16")
            else:
                df[col] = df[col].astype("float32")
            # float64는 유지되지 않도록 float32로 축소
    print(df.info())

    return df

In [ ]:
# 모델 평가 함수 정의
def evaluate_model(model, X_train, X_test, y_train, y_test):
    """
    모델 평가 함수
    
    전달인자 설명:
    model: 학습된 머신러닝 모델
    X_train: 학습 데이터 피처
    X_test: 테스트 데이터 피처
    y_train: 학습 데이터 타겟
    y_test: 테스트 데이터 타겟
    """
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
    import numpy as np

    # 테스트 데이터에 대한 예측 수행
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # 평가
    train_mse = mean_squared_error(y_train, y_pred_train)
    test_mse = mean_squared_error(y_test, y_pred_test)
    train_rmse = np.sqrt(train_mse)
    test_rmse = np.sqrt(test_mse)
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)

    print(f"훈련 데이터 MAE: {train_mae:.2f}")
    print(f"테스트 데이터 MAE: {test_mae:.2f}")
    print("*" * 50)
    print(f"훈련 데이터 MSE: {train_mse:.2f}")
    print(f"테스트 데이터 MSE: {test_mse:.2f}")
    print("*" * 50)
    print(f"훈련 데이터 RMSE: {train_rmse:.2f}")
    print(f"테스트 데이터 RMSE: {test_rmse:.2f}")
    print("*" * 50)
    print(f"훈련 데이터 R²: {train_r2:.2f}")
    print(f"테스트 데이터 R²: {test_r2:.2f}")
    print("*" * 50)

    # 평균값 계산
    train_mean = y_train.mean()
    test_mean = y_test.mean()

    print(f"훈련 데이터 평균값: {train_mean}")
    print(f"테스트 데이터 평균값: {test_mean}")

    # MAE 대비 비율 계산
    train_mae_ratio = (train_mae / train_mean) * 100
    test_mae_ratio = (test_mae / test_mean) * 100

    print(f"훈련 데이터 MAE 비율: {train_mae_ratio:.2f}%")
    print(f"테스트 데이터 MAE 비율: {test_mae_ratio:.2f}%")

    # 평가 결과 반환
    return {
        'train_mse': train_mse,
        'test_mse': test_mse,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'train_mae': train_mae,
        'test_mae': test_mae,
        'train_mae_ratio': train_mae_ratio,
        'test_mae_ratio': test_mae_ratio
    }

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

def preprocess_sales_data(data, target_column="월매출", category_column="업종명", filter_value="한식음식점", one_hot_column="행정동", test_size=0.2, random_state=42):
    """
    판매 데이터 전처리: 데이터 필터링, 원핫 인코딩, 타겟/피처 분리, 학습/테스트 데이터 분할.

    Parameters:
        data (pd.DataFrame): 원본 데이터프레임.
        target_column (str): 타겟 변수 열 이름. 기본값: "월매출".
        category_column (str): 필터링 기준 열 이름. 기본값: "업종명".
        filter_value (str): 필터링할 값. 기본값: "한식음식점".
        one_hot_column (str): 원핫 인코딩을 수행할 열 이름. 기본값: "행정동".
        test_size (float): 테스트 데이터 비율. 기본값: 0.2.
        random_state (int): 랜덤 시드 값. 기본값: 42.

    Returns:
        tuple: (X_train, X_test, y_train, y_test) 학습/테스트 데이터 분할 결과.
    """
    # 필터링 및 특정 열 삭제
    df_filtered = data[data[category_column] == filter_value].drop(category_column, axis=1)

    # 원핫 인코딩 수행
    df_encoded = pd.get_dummies(df_filtered, columns=[one_hot_column])

    # 타겟과 피처 분리
    target = df_encoded[target_column]
    features = df_encoded.drop(target_column, axis=1)

    # 학습/테스트 데이터로 분할
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=test_size, random_state=random_state)

    # 피처와 타겟의 형태 출력
    print("Features shape:", features.shape)
    print("Target shape:", target.shape)

    return X_train, X_test, y_train, y_test
